# Delta Live Tables

## Bronze Layer Tables

In [0]:
use catalog hive_metastore

In [0]:
-- 파일 폴더에 새로 들어오는 주문 데이터를 자동으로 읽어 항상 최신 상태의 테이블로 유지하는 정의입니다.
create or refresh streaming live table orders_raw  -- “실시간으로 업데이트되는 테이블을 만들겠다”
comment "The raw books orders, ingested from orders-raw"
as select * from cloud_files(                                   -- Auto Loader의 SQL 버전
    "dbfs:/mnt/demo-datasets/bookstore/orders-raw", "json",     -- 새 JSON 파일만 자동으로 읽음 / 이미 읽은 파일은 다시 안 읽음
    map("cloudFiles.inferColumnTypes", "true"))                 -- 스키마를 내가 안 써도 알아서 맞춰

## Customers

In [0]:
CREATE OR REFRESH LIVE TABLE customers
COMMENT "The customers lookup table, ingested from customers-json"
AS SELECT * FROM json.`dbfs:/mnt/demo-datasets/bookstore/customers-json`

## Silver Layer Tables
### orders_cleaned

In [0]:
create or refresh streaming live table orders_cleaned (
  constraint valid_order_number expect (order_id is not null) on violation drop row
)
comment "The cleaned books orders with valid order_id"
as select order_id, quantity, o.customer_id, c.profile:first_name as f_name, c.profile:last_name as l_name,
          cast(from_unixtime(order_timestamp, 'yyyy-MM-dd HH:mm:ss') AS timestamp) order_timestamp, o.books,
          c.profile:address:country as country
  from stream(live.orders_raw) o 
  -- left join live.customers c
  --   on o.customer_id = c.customer_id

>> Constraint violation

| **`ON VIOLATION`** | Behavior |
| --- | --- |
| **`DROP ROW`** | Discard records that violate constraints |
| **`FAIL UPDATE`** | Violated constraint causes the pipeline to fail  |
| Omitted | Records violating constraints will be kept, and reported in metrics |

## Gold Tables

In [0]:
CREATE OR REFRESH LIVE TABLE cn_daily_customer_books
COMMENT "Daily number of books per customer in China"
AS
  SELECT customer_id, f_name, l_name, date_trunc("DD", order_timestamp) order_date, sum(quantity) books_counts
  FROM LIVE.orders_cleaned
  WHERE country = "China"
  GROUP BY customer_id, f_name, l_name, date_trunc("DD", order_timestamp)

In [0]:
CREATE OR REFRESH LIVE TABLE fr_daily_customer_books
COMMENT "Daily number of books per customer in France"
AS
  SELECT customer_id, f_name, l_name, date_trunc("DD", order_timestamp) order_date, sum(quantity) books_counts
  FROM LIVE.orders_cleaned
  WHERE country = "France"
  GROUP BY customer_id, f_name, l_name, date_trunc("DD", order_timestamp)